# Setting up SQLite environment

In [ ]:

import pandas as pd
import sqlite3
import os

# 1. Configuration – adjust QUARTERS if you only want one
DATA_DIR = '../data'
QUARTERS = ['24Q4', '25Q1']      # or ['25Q1'] to just load Q1 2025
TABLES = ['DEMO', 'DRUG', 'REAC']
DB_PATH = os.path.join(DATA_DIR, 'faers.db')

# 2. Connect to your SQLite DB
conn = sqlite3.connect(DB_PATH)

# 3. Loop through quarters and tables
for quarter in QUARTERS:
    year = 2000 + int(quarter[:2])        # e.g. 2000+24 = 2024
    period = quarter[2:]                  # 'Q4' or 'Q1'
    folder = f"FAERS{period}_{year}"      # e.g. 'FAERSQ4_2024'
    
    for tbl in TABLES:
        path = os.path.join(DATA_DIR, folder, 'ASCII', f"{tbl}{quarter}.txt")
        
        # 4. Read the raw file into pandas
        df = pd.read_csv(
            path,
            sep='$',
            encoding='latin-1',
            dtype=str,
            low_memory=False
        )
        
        # 5. Tag rows with their quarter
        df['quarter'] = quarter
        
        # 6. Write to SQLite
        table_name = f"{tbl.lower()}{quarter.lower()}" 
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        
        #print(f"Loaded {table_name}: {df.shape[0]} rows")
conn.close()

| Data Type  | Column name     | Description                                                        |
|---------|--------------|--------------------------------------------------------------------|
| DEMO    | primaryid     | Unique ID for a single report                                     |
| DEMO    | caseid        | Case identifier (can be shared across multiple reports)            |
| DEMO    | age           | Patient age                                                       |
| DEMO    | age_cod       | Units for age (e.g., YR, MON, DY)                                 |
| DEMO    | age_grp       | Age group (e.g., A, C)                                            |
| DEMO    | sex           | Gender (M, F, U)                                                  |
| DEMO    | wt            | Weight                                                            |
| DEMO    | wt_cod        | Units for weight (e.g., KG, LB)                                   |
| DEMO    | rept_dt       | Date the report was filed                                         |
| DEMO    | occr_country  | Country where the reaction occurred                               |
| DRUG    | primaryid     | Links to DEMO and REAC                                            |
| DRUG    | drugname      | Name of the drug reported                                         |
| DRUG    | role_cod      | Role of the drug (PS = primary suspect, SS = secondary suspect)   |
| DRUG    | route         | Route of administration (oral, intravenous, etc.)                 |
| DRUG    | dose_vbm      | Dose given                                                        |
| DRUG    | drug_seq      | Sequence of drug in multi-drug report                             |
| REAC    | primaryid     | Links to DEMO and REAC                                            |
| REAC    | pt            | Preferred Term – the actual adverse event (e.g., nausea, death)   |
| REAC    | outc_cod      | Outcome code (e.g., death, hospitalization)                       |

In [19]:
# 7. Check data query
def sql(query):
    return pd.read_sql_query(query, conn)

sql("""
SELECT
    (SELECT COUNT(primaryid) FROM reac24q4) AS total_reports_24q4,
    (SELECT COUNT(primaryid) FROM reac25q1) AS total_reports_25q1,
    (SELECT COUNT(primaryid) FROM demo24q4) AS total_demo_24q4,
    (SELECT COUNT(primaryid) FROM demo25q1) AS total_demo_25q
""")

,total_reports_24q4,total_reports_25q1,total_demo_24q4,total_demo_25q
0,1472750,1432926,410849,400514


In [ ]:
git rm -r --cached .venv/ data
git rm --cached *.db *.txt *.zip
git add .gitignore
git commit -m "Remove venv & raw FAERS data from tracking"